In [1]:
!pip install transformers==4.34.0
!pip install sentencepiece==0.1.99
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 92.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.8/395.8 kB 26.4 MB/

In [2]:
import gradio as gr
import torch
import gc

# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


device = "cuda:0" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("lmsys/fastchat-t5-3b-v1.0")
model = AutoModelForSeq2SeqLM.from_pretrained("lmsys/fastchat-t5-3b-v1.0")
model = model.to(device)

input_text = 'Translate from english to german: How old are you'
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
print('input_text', input_text)
print('input_ids', input_ids)
outputs = model.generate(input_ids, max_length=500)
print('outputs', outputs)
text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("outputs text:", text)

def greet0(human_inputs):

    system_message = "A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions."

    input_text = f"{system_message}\n### Human: {human_inputs}\n### Assistant: "
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
    print('input_text', input_text)
    print('input_ids', input_ids)
    outputs = model.generate(input_ids, max_length=500)
    print('outputs', outputs)
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("outputs text:", text)
    return text


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


input_text Translate from english to german: How old are you
input_ids tensor([[30355,    15, 32103,    45, 32103, 22269, 32103,    12, 32103, 13692,
            10, 32103,   571, 32103,   625, 32103,    33, 32103,    25,     1]])
outputs tensor([[    0,  2739, 32106,  4445, 32106,   436, 32106,   292,    58, 32103,
             1]])
outputs text: <pad> Wie  alt  sind  Sie?



In [ ]:

# stream out

yield_tokens = []


def greet_stream(human_inputs):
    system_message = "A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions."
    input_text = f"{system_message}\n### Human: {human_inputs}\n### Assistant: "
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

    max_length = 200
    encoder_output = model.encoder(
        input_ids=input_ids
    )['last_hidden_state']  # (1, len, d_model)

    gen_ids = torch.tensor([[0]]).to(device)
    past_key_values = None
    yield_tokens = []
    for i in range(max_length):
        out = model.decoder(
            input_ids=gen_ids,
            encoder_hidden_states=encoder_output,
            use_cache=True,
            past_key_values=past_key_values,
        )
        past_key_values = out.past_key_values
        last_hidden_state = out.last_hidden_state  # (1, 1, d_model)
        lm_logits = model.lm_head(last_hidden_state)  # (1, 1, len_dict)
        values, indices = lm_logits[0].topk(2)
        # print(tokenizer.convert_ids_to_tokens(indices[0]))
        gen_ids = torch.index_select(indices, 1, torch.tensor([0]).to(device))
        yield_tokens.append(tokenizer.decode(
            gen_ids[0], skip_special_tokens=True))
        if torch.equal(gen_ids[0], torch.tensor([1]).to(device)):
          # eos token
          break
        gc.collect()
        torch.cuda.empty_cache()
        yield "".join(yield_tokens)

    # stream
    return "".join(yield_tokens)


iface = gr.Interface(fn=greet_stream, inputs=[gr.Textbox(
    label="Text 1",
    info="Initial text",
    lines=3,
    value="Who are you?",
)], outputs="text")

iface.queue()
iface.launch(debug=True)



Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://fd828403b7cc3062c4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
